In [80]:
import numpy as np
import matplotlib.pyplot as plt 

In [81]:
def gauss_jordan(A, B):
    for i in range(len(A[0])):
        #partial pivoting
        besti = i
        for j in range(i+1,len(A)):
            if A[i][i] < A[j][i]:
                besti = j
        for k in range(len(A[0])):
            A[i][k], A[besti][k] = A[besti][k], A[i][k]
        #counting
        for j in range(len(A)):
            if j != i: 
                d = -A[j][i]/A[i][i]
                for k in range(i, len(A[i])):
                    A[j][k] += d*A[i][k]
                B[j] += d*B[i]
        for k in range(i,len(A[0])):
            A[i][k] /= A[i][i]

A = np.random.rand(5,5)
B = np.random.rand(5)
gauss_jordan(A, B)
print(A);
print(B);

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  2.77555756e-17
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.55111512e-17  0.00000000e+00
   1.00000000e+00]]
[ 0.10344752  0.30151002 -0.70117536  0.02745554 -2.01585922]


In [82]:
def LU(A):
    for i in range(len(A[0])):
        #counting
        for j in range(i+1, len(A)):
            d = -A[j][i]/A[i][i]
            for k in range(i, len(A[i])):
                A[j][k] += d*A[i][k]
            A[j][i] = -d;
    L = np.zeros((len(A), len(A)))
    U = np.zeros((len(A), len(A)))
    for i in range(len(A[0])):
        for j in range(len(A)):
            if i==j:
                L[i][j] = 1
                U[i][j] = A[i][j]
            elif i < j:
                L[i][j] = 0
                U[i][j] = A[i][j]
            else:
                L[i][j] = A[i][j]
                U[i][j] = 0
    return L, U
                

In [84]:
A = np.random.rand(5,5)
print(A)
L, U = LU(A)
print(L)
print(U)
print(L @ U)

[[0.32500587 0.17920482 0.71644379 0.97529472 0.58824043]
 [0.8701252  0.82848344 0.98579279 0.68777707 0.44659057]
 [0.5921283  0.63591649 0.13650044 0.26932602 0.32860939]
 [0.50186624 0.4492092  0.1532319  0.53225237 0.04013255]
 [0.1315458  0.32638895 0.90828578 0.14018336 0.77963688]]
[[ 1.          0.          0.          0.          0.        ]
 [ 2.67725998  1.          0.          0.          0.        ]
 [ 1.82190036  0.88734799  1.          0.          0.        ]
 [ 1.54417595  0.49464493  1.44045673  1.          0.        ]
 [ 0.404749    0.72799523 -3.79800017 -6.15059466  1.        ]]
[[ 0.32500587  0.17920482  0.71644379  0.97529472  0.58824043]
 [ 0.          0.34870556 -0.9323135  -1.92334044 -1.12828198]
 [ 0.          0.         -0.34150226  0.19910848  0.25807269]
 [ 0.          0.          0.         -0.30921083 -0.68185775]
 [ 0.          0.          0.          0.         -1.8507395 ]]
[[0.32500587 0.17920482 0.71644379 0.97529472 0.58824043]
 [0.8701252  0.8284